---
## 1. 📦 Setup & Installation

In [2]:
# Check if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")

# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Running in Google Colab: True
PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


In [3]:
# Install required packages
!pip install -q pytorch-lightning timm albumentations opencv-python-headless scikit-learn pandas tqdm matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 22.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.9 MB/s eta 0:00:00


In [4]:
# Core imports
import os
import sys
import time
import logging
from pathlib import Path
from datetime import datetime
from typing import Optional, Dict, List, Tuple, Union, Callable

# Data handling
import numpy as np
import pandas as pd

# Image processing
import cv2
from PIL import Image

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger

# Model
import timm

# Augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Metrics
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

# Optimization
from scipy.optimize import minimize

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Progress
from tqdm.auto import tqdm

# Warnings
import warnings
warnings.filterwarnings('ignore')

# ============================================
# GPU & Performance Optimizations
# ============================================

# Set device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {DEVICE}")

if torch.cuda.is_available():
    # Enable cuDNN auto-tuner for faster training
    torch.backends.cudnn.benchmark = True
    # Enable TF32 for Ampere GPUs (faster matmul)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print(f"⚡ cuDNN benchmark: ENABLED")
    print(f"⚡ TF32 mode: ENABLED (for Ampere GPUs)")

# Set random seeds for reproducibility
def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Note: We keep deterministic=False for speed with benchmark=True
    pl.seed_everything(seed, workers=True)

seed_everything(42)

print("✅ All imports successful!")

INFO:lightning_fabric.utilities.seed:Seed set to 42


🖥️ Using device: cuda
⚡ cuDNN benchmark: ENABLED
⚡ TF32 mode: ENABLED (for Ampere GPUs)
✅ All imports successful!


---
## 2. 📁 Data Loading (Kaggle API)

Since VS Code's remote connection to Colab doesn't show the Google Drive auth popup, we'll download from Kaggle instead.

**First time setup:**
1. Go to [kaggle.com](https://kaggle.com) → Your Profile → Account → Create API Token
2. Download `kaggle.json`
3. Upload it using the cell below

In [5]:
# ============================================
# SETUP KAGGLE API KEY
# ============================================
# Your kaggle.json is at C:\Users\kevin\.kaggle\kaggle.json
# Open that file and paste the contents below

import os
import json

# PASTE YOUR KAGGLE CREDENTIALS HERE (open C:\Users\kevin\.kaggle\kaggle.json and copy contents)
kaggle_credentials = {"username":"kevinpaulg","key":""}

# Write to Colab
os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_credentials, f)
os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("✅ Kaggle API configured!")
print(f"   Username: {kaggle_credentials.get('username', 'NOT SET')}")

✅ Kaggle API configured!
   Username: kevinpaulg


In [13]:
# ============================================
# DOWNLOAD APTOS DATASET FROM KAGGLE
# ============================================
# This downloads ~13GB of data - be patient!

import os

# Create directories
os.makedirs("/content/data/aptos", exist_ok=True)

print("📥 Downloading APTOS 2019 dataset from Kaggle...")
print("⚠️  This is a large dataset (~13GB) - please wait!")
print("="*50)

# Check if already downloaded
zip_path = "/content/data/aptos2019-blindness-detection.zip"
csv_exists = os.path.exists("/content/data/aptos/train.csv")

if csv_exists:
    print("✅ Dataset already downloaded and extracted!")
else:
    # Download with verbose output
    !kaggle competitions download -c aptos2019-blindness-detection -p /content/data
    
    # Check if download succeeded
    if os.path.exists(zip_path):
        print("\n✅ Download complete!")
        print("📦 Extracting dataset (this may take a minute)...")
        !unzip -q -o {zip_path} -d /content/data/aptos
        print("✅ Extraction complete!")
        
        # Clean up zip to save space
        !rm {zip_path}
        print("🗑️ Cleaned up zip file to save disk space")
    else:
        print("\n❌ Download failed! The zip file was not created.")
        print("   Please try running this cell again.")

# Verify contents
print("\n📁 Dataset contents:")
!ls -la /content/data/aptos/

📥 Downloading APTOS 2019 dataset from Kaggle...
⚠️  This is a large dataset (~13GB) - please wait!
^C

❌ Download failed! The zip file was not created.
   Please try running this cell again.

📁 Dataset contents:
total 8
drwxr-xr-x 2 root root 4096 Dec  2 12:40 .
drwxr-xr-x 3 root root 4096 Dec  2 12:40 ..
^C

❌ Download failed! The zip file was not created.
   Please try running this cell again.

📁 Dataset contents:
total 8
drwxr-xr-x 2 root root 4096 Dec  2 12:40 .
drwxr-xr-x 3 root root 4096 Dec  2 12:40 ..


In [14]:
# ============================================
# EXTRACT ZIP FROM GOOGLE DRIVE
# ============================================
# Update this path to where your zip file is in Google Drive!

import os

# UPDATE THIS PATH to your actual zip location in Google Drive
DRIVE_ZIP = "/content/drive/MyDrive/aptos2019-blindness-detection.zip"

# Alternative common locations - uncomment the right one:
# DRIVE_ZIP = "/content/drive/MyDrive/Datasets/aptos2019-blindness-detection.zip"
# DRIVE_ZIP = "/content/drive/MyDrive/ML/aptos2019-blindness-detection.zip"

print(f"Looking for: {DRIVE_ZIP}")

if os.path.exists(DRIVE_ZIP):
    print("✅ Found zip file!")
    os.makedirs("/content/data/aptos", exist_ok=True)
    
    print("📦 Extracting (this takes ~1 minute)...")
    !unzip -q -o "{DRIVE_ZIP}" -d /content/data/aptos
    
    print("✅ Extraction complete!")
    !ls -la /content/data/aptos/
else:
    print(f"❌ Zip not found at: {DRIVE_ZIP}")
    print("\n📂 Let's find it. Contents of your Drive:")
    !ls -la /content/drive/MyDrive/ | head -20
    print("\n💡 Update DRIVE_ZIP path above to match your zip file location")

Looking for: /content/drive/MyDrive/aptos2019-blindness-detection.zip
❌ Zip not found at: /content/drive/MyDrive/aptos2019-blindness-detection.zip

📂 Let's find it. Contents of your Drive:
ls: cannot access '/content/drive/MyDrive/': No such file or directory

💡 Update DRIVE_ZIP path above to match your zip file location

💡 Update DRIVE_ZIP path above to match your zip file location


In [12]:
# ============================================
# CONFIGURATION & DATA LOADING
# ============================================
from pathlib import Path

class Config:
    """Training configuration optimized for GPU performance."""
    # Paths - Data extracted from Google Drive
    DATA_DIR = Path("/content/data/aptos")
    TRAIN_CSV = "train.csv"
    TRAIN_IMAGES = "train_images"  # Raw images folder
    PROCESSED_IMAGES = "processed"  # Preprocessed images (will be created)
    CHECKPOINT_DIR = Path("/content/checkpoints")
    LOG_DIR = Path("/content/logs")
    
    # Dataset
    NUM_CLASSES = 5
    CLASS_NAMES = ["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"]
    
    # Model
    BACKBONE = "tf_efficientnet_b5_ns"
    IMAGE_SIZE = 456
    POOLING = "gem"
    HEAD_TYPE = "regression"
    HEAD_DROPOUT = 0.5
    
    # Training - Optimized for GPU
    EPOCHS = 30
    BATCH_SIZE = 16  # Adjust based on GPU (T4: 16, V100: 32)
    ACCUMULATE_GRAD = 2
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    PATIENCE = 5
    VAL_SPLIT = 0.2
    
    # Data Loading
    NUM_WORKERS = 2  # Colab works better with 2
    PREFETCH_FACTOR = 2
    PERSISTENT_WORKERS = True
    
    # GPU Optimizations
    PRECISION = "16-mixed" if torch.cuda.is_available() else 32
    PIN_MEMORY = torch.cuda.is_available()
    
    # Preprocessing
    BEN_GRAHAM_SIZE = 512
    USE_PREPROCESSING = True
    
    # Seed
    SEED = 42

config = Config()

# Create directories
config.CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
config.LOG_DIR.mkdir(parents=True, exist_ok=True)

# ============================================
# VERIFY DATA
# ============================================
csv_path = config.DATA_DIR / config.TRAIN_CSV
raw_images_path = config.DATA_DIR / config.TRAIN_IMAGES

print("📊 Data Verification:")

# Load CSV
if csv_path.exists():
    df = pd.read_csv(csv_path)
    print(f"✅ Labels: {len(df)} samples")
    print(f"   Classes: {df['diagnosis'].value_counts().to_dict()}")
else:
    print(f"❌ Labels not found: {csv_path}")
    df = None

# Check raw images
if raw_images_path.exists():
    raw_files = list(raw_images_path.glob("*.png")) + list(raw_images_path.glob("*.jpg"))
    print(f"✅ Raw images: {len(raw_files)} files in train_images/")
else:
    print(f"❌ Raw images not found: {raw_images_path}")

# Check for processed (may not exist yet)
processed_path = config.DATA_DIR / config.PROCESSED_IMAGES
if processed_path.exists():
    proc_files = list(processed_path.glob("*.png"))
    print(f"✅ Processed images: {len(proc_files)} files")
else:
    print(f"ℹ️  No processed/ folder yet (will be created)")

print("\n" + "="*50)
print("🎉 Data ready for preprocessing!" if df is not None else "⚠️ Please check your data")
print("="*50)

📊 Data Verification:
❌ Labels not found: /content/data/aptos/train.csv
❌ Raw images not found: /content/data/aptos/train_images
ℹ️  No processed/ folder yet (will be created)

⚠️ Please check your data


In [8]:
# Configuration
import os

# Determine base directory based on environment
if IN_COLAB:
    # In Colab, data should be at /content/data/aptos after running data setup cell
    _base_dir = Path("/content")
else:
    # Local: handle running from notebooks/ or project root
    _notebook_dir = Path(os.getcwd())
    if _notebook_dir.name == "notebooks":
        _base_dir = _notebook_dir.parent
    else:
        _base_dir = _notebook_dir

class Config:
    """Training configuration optimized for GPU performance."""
    # Paths - Dynamically resolved based on environment
    DATA_DIR = _base_dir / "data" / "aptos"
    TRAIN_CSV = "train.csv"
    TRAIN_IMAGES = "train_images"
    PROCESSED_IMAGES = "processed"  # Your preprocessed images folder
    CHECKPOINT_DIR = _base_dir / "checkpoints"
    LOG_DIR = _base_dir / "logs"
    
    # Dataset
    NUM_CLASSES = 5
    CLASS_NAMES = ["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"]
    
    # Model
    BACKBONE = "tf_efficientnet_b5_ns"  # Noisy Student pretrained
    IMAGE_SIZE = 456
    POOLING = "gem"  # Generalized Mean Pooling
    HEAD_TYPE = "regression"
    HEAD_DROPOUT = 0.5
    
    # Training - Optimized for GPU
    EPOCHS = 30
    BATCH_SIZE = 16  # Adjust based on GPU memory (V100: 32, T4: 16, 3090: 24)
    ACCUMULATE_GRAD = 2  # Effective batch size = BATCH_SIZE * ACCUMULATE_GRAD
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    PATIENCE = 5
    VAL_SPLIT = 0.2
    
    # Data Loading - Optimized for fast I/O
    NUM_WORKERS = 4 if torch.cuda.is_available() else 2  # More workers for GPU
    PREFETCH_FACTOR = 2  # Prefetch batches in parallel
    PERSISTENT_WORKERS = True  # Keep workers alive between epochs
    
    # GPU Optimizations
    PRECISION = "16-mixed" if torch.cuda.is_available() else 32  # Mixed precision
    PIN_MEMORY = torch.cuda.is_available()  # Pin memory for faster GPU transfer
    
    # Preprocessing
    BEN_GRAHAM_SIZE = 512
    USE_PREPROCESSING = True  # We have preprocessed images
    
    # Seed
    SEED = 42

config = Config()

# Create directories
config.DATA_DIR.mkdir(parents=True, exist_ok=True)
config.CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
config.LOG_DIR.mkdir(parents=True, exist_ok=True)

print("📋 Configuration:")
print(f"  - Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"  - Base Directory: {_base_dir}")
print(f"  - Data Directory: {config.DATA_DIR}")
print(f"  - Backbone: {config.BACKBONE}")
print(f"  - Image Size: {config.IMAGE_SIZE}")
print(f"  - Batch Size: {config.BATCH_SIZE} (effective: {config.BATCH_SIZE * config.ACCUMULATE_GRAD})")
print(f"  - Epochs: {config.EPOCHS}")
print(f"  - Learning Rate: {config.LEARNING_RATE}")
print(f"  - Precision: {config.PRECISION}")
print(f"  - Num Workers: {config.NUM_WORKERS}")
print(f"  - Pin Memory: {config.PIN_MEMORY}")

📋 Configuration:
  - Environment: Google Colab
  - Base Directory: /content
  - Data Directory: /content/data/aptos
  - Backbone: tf_efficientnet_b5_ns
  - Image Size: 456
  - Batch Size: 16 (effective: 32)
  - Epochs: 30
  - Learning Rate: 0.0001
  - Precision: 16-mixed
  - Num Workers: 4
  - Pin Memory: True


In [11]:
# Visualize class distribution
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    class_counts = df['diagnosis'].value_counts().sort_index()
    colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, 5))
    
    axes[0].bar(range(5), class_counts.values, color=colors)
    axes[0].set_xticks(range(5))
    axes[0].set_xticklabels(config.CLASS_NAMES, rotation=45, ha='right')
    axes[0].set_xlabel('DR Grade')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Class Distribution')
    
    for i, v in enumerate(class_counts.values):
        axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')
    
    # Pie chart
    axes[1].pie(class_counts.values, labels=config.CLASS_NAMES, autopct='%1.1f%%', 
                colors=colors, explode=[0.05]*5)
    axes[1].set_title('Class Distribution (%)')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Class Distribution:")
    for i, name in enumerate(config.CLASS_NAMES):
        count = class_counts.get(i, 0)
        pct = count / len(df) * 100
        print(f"  {i} - {name}: {count} ({pct:.1f}%)")
else:
    print("⚠️ DataFrame 'df' is not loaded!")
    print("   Please run the data loading cells first (Cells 6-8)")
    print("\n   To debug, run this:")
    print("   !ls -la /content/data/aptos/")

⚠️ DataFrame 'df' is not loaded!
   Please run the data loading cells first (Cells 6-8)

   To debug, run this:
   !ls -la /content/data/aptos/


In [ ]:
# ============================================
# VERIFY RAW DATA IS LOADED
# ============================================
import os

# Get current directory and resolve paths
notebook_dir = Path(os.getcwd())
print(f"Current working directory: {notebook_dir}")

# If running from notebooks folder, adjust the base path
if notebook_dir.name == "notebooks":
    base_dir = notebook_dir.parent
else:
    base_dir = notebook_dir

# Update config paths to absolute
config.DATA_DIR = base_dir / "data" / "aptos"
config.CHECKPOINT_DIR = base_dir / "checkpoints"
config.LOG_DIR = base_dir / "logs"

# Define paths
train_csv_path = config.DATA_DIR / config.TRAIN_CSV
train_images_dir = config.DATA_DIR / config.TRAIN_IMAGES  # Raw images
processed_dir = config.DATA_DIR / config.PROCESSED_IMAGES  # Will be created

print(f"\n📁 Data Paths:")
print(f"   - Data directory: {config.DATA_DIR}")
print(f"   - Labels file: {train_csv_path}")
print(f"   - Raw images: {train_images_dir}")
print(f"   - Processed output: {processed_dir}")

# Check raw images (this is what we need!)
print(f"\n📊 Raw Data Check:")
if train_images_dir.exists():
    raw_images = list(train_images_dir.glob("*.png")) + list(train_images_dir.glob("*.jpg"))
    print(f"   ✅ Found {len(raw_images)} raw images in train_images/")
    
    # Show sample raw image
    if raw_images:
        sample_img_path = raw_images[0]
        sample_img = cv2.imread(str(sample_img_path))
        sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 2, 1)
        plt.imshow(sample_img)
        plt.title(f"Sample RAW Image: {sample_img_path.name}\nShape: {sample_img.shape}")
        plt.axis('off')
        
        # Preview what preprocessing will do
        preprocessor = BenGrahamPreprocessor(output_size=config.BEN_GRAHAM_SIZE)
        processed_preview = preprocessor(cv2.cvtColor(sample_img, cv2.COLOR_RGB2BGR))
        processed_preview = cv2.cvtColor(processed_preview, cv2.COLOR_BGR2RGB)
        
        plt.subplot(1, 2, 2)
        plt.imshow(processed_preview)
        plt.title(f"After Ben Graham Preprocessing\nShape: {processed_preview.shape}")
        plt.axis('off')
        
        plt.suptitle("Raw → Processed Preview", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
else:
    print(f"   ❌ Raw images not found at {train_images_dir}")
    print(f"      Please run the data acquisition cell above!")

# Check if processed folder already exists (from previous run)
if processed_dir.exists():
    processed_images = list(processed_dir.glob("*.png"))
    print(f"   ℹ️ Found {len(processed_images)} already-processed images in processed/")
else:
    print(f"   ℹ️ No processed/ folder yet - will be created during preprocessing")

Current working directory: /content
Data directory: /content/data/aptos
Train images directory: /content/data/aptos/train_images
❌ Image directory not found at /content/data/aptos/train_images
   Please check if the data is in the correct location.


---
## 3. 🔧 Ben Graham Preprocessing (Raw → Processed)

**This section transforms raw fundus images into preprocessed images for training.**

The Ben Graham preprocessing technique:
1. **Crops** the fundus image to remove black borders
2. **Subtracts local Gaussian blur** for luminosity normalization  
3. **Applies circular mask** to remove edge artifacts

This significantly improves model performance by normalizing image quality across different cameras/settings.

**Input:** `train_images/` (raw images)  
**Output:** `processed/` (preprocessed images ready for training)

In [ ]:
class BenGrahamPreprocessor:
    """
    Ben Graham preprocessing for fundus images.
    
    Formula: I_processed = α * I_original + β * G(I_original, σ) + γ
    Default: α=4, β=-4, γ=128 (acts as high-pass filter)
    """
    
    def __init__(
        self,
        output_size: int = 512,
        alpha: float = 4.0,
        beta: float = -4.0,
        gamma: float = 128.0,
        sigma_ratio: float = 0.1,
        crop_black_margin: bool = True,
        apply_circular_mask: bool = True,
    ):
        self.output_size = output_size
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.sigma_ratio = sigma_ratio
        self.crop_black_margin = crop_black_margin
        self.apply_circular_mask = apply_circular_mask
        
        sigma = int(output_size * sigma_ratio)
        self.kernel_size = sigma * 2 + 1
        self.sigma = sigma
    
    def _find_eye_region(self, image: np.ndarray) -> Tuple[int, int, int, int]:
        """Find bounding box of the eye region."""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            return 0, 0, image.shape[1], image.shape[0]
        
        largest_contour = max(contours, key=cv2.contourArea)
        return cv2.boundingRect(largest_contour)
    
    def _crop_to_square(self, image: np.ndarray) -> np.ndarray:
        """Crop image to square centered on eye."""
        if not self.crop_black_margin:
            h, w = image.shape[:2]
            min_dim = min(h, w)
            start_x, start_y = (w - min_dim) // 2, (h - min_dim) // 2
            return image[start_y:start_y + min_dim, start_x:start_x + min_dim]
        
        x, y, w, h = self._find_eye_region(image)
        center_x, center_y = x + w // 2, y + h // 2
        
        size = int(max(w, h) * 1.05)
        half_size = size // 2
        
        img_h, img_w = image.shape[:2]
        x1 = max(0, center_x - half_size)
        y1 = max(0, center_y - half_size)
        x2 = min(img_w, center_x + half_size)
        y2 = min(img_h, center_y + half_size)
        
        cropped = image[y1:y2, x1:x2]
        
        crop_h, crop_w = cropped.shape[:2]
        if crop_h != crop_w:
            max_side = max(crop_h, crop_w)
            square = np.zeros((max_side, max_side, 3), dtype=np.uint8)
            y_offset = (max_side - crop_h) // 2
            x_offset = (max_side - crop_w) // 2
            square[y_offset:y_offset + crop_h, x_offset:x_offset + crop_w] = cropped
            cropped = square
        
        return cropped
    
    def _apply_gaussian_filter(self, image: np.ndarray) -> np.ndarray:
        """Apply Ben Graham Gaussian blur subtraction."""
        blurred = cv2.GaussianBlur(image, (self.kernel_size, self.kernel_size), self.sigma)
        processed = cv2.addWeighted(image, self.alpha, blurred, self.beta, self.gamma)
        return np.clip(processed, 0, 255).astype(np.uint8)
    
    def _apply_circular_mask_fn(self, image: np.ndarray) -> np.ndarray:
        """Apply circular mask."""
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        radius = min(center) - 2
        
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.circle(mask, center, radius, 255, -1)
        
        result = image.copy()
        result[mask == 0] = 0
        return result
    
    def __call__(self, image: np.ndarray) -> np.ndarray:
        """Apply full preprocessing pipeline."""
        cropped = self._crop_to_square(image)
        resized = cv2.resize(cropped, (self.output_size, self.output_size), interpolation=cv2.INTER_LINEAR)
        processed = self._apply_gaussian_filter(resized)
        
        if self.apply_circular_mask:
            processed = self._apply_circular_mask_fn(processed)
        
        return processed

print("✅ BenGrahamPreprocessor defined!")

✅ BenGrahamPreprocessor defined!


In [ ]:
# Demonstrate preprocessing on sample images
if 'df' in dir() and train_images_dir.exists():
    preprocessor = BenGrahamPreprocessor(output_size=config.BEN_GRAHAM_SIZE)
    
    # Get one sample per class
    fig, axes = plt.subplots(5, 3, figsize=(15, 25))
    
    for class_idx in range(5):
        # Get sample from this class
        sample = df[df['diagnosis'] == class_idx].iloc[0]
        img_name = sample['id_code']
        
        # Find image file
        img_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            potential_path = train_images_dir / f"{img_name}{ext}"
            if potential_path.exists():
                img_path = potential_path
                break
        
        if img_path:
            # Load and process
            img = cv2.imread(str(img_path))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            processed = preprocessor(img)
            processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
            
            # Original
            axes[class_idx, 0].imshow(img_rgb)
            axes[class_idx, 0].set_title(f"Original - {config.CLASS_NAMES[class_idx]}")
            axes[class_idx, 0].axis('off')
            
            # Processed
            axes[class_idx, 1].imshow(processed_rgb)
            axes[class_idx, 1].set_title(f"Ben Graham Processed")
            axes[class_idx, 1].axis('off')
            
            # Histogram
            for c, color in enumerate(['red', 'green', 'blue']):
                axes[class_idx, 2].hist(processed_rgb[:,:,c].ravel(), bins=50, 
                                        alpha=0.5, color=color, label=color.upper())
            axes[class_idx, 2].set_title('Color Histogram')
            axes[class_idx, 2].legend()
    
    plt.tight_layout()
    plt.show()
    print("\n✅ Preprocessing demonstration complete!")

In [ ]:
# ============================================
# PREPROCESS ALL RAW IMAGES → PROCESSED FOLDER
# ============================================
# This is the KEY step: Convert raw train_images/ to processed/

def preprocess_all_images(df, input_dir, output_dir, preprocessor, overwrite=False):
    """
    Preprocess all raw images and save to processed folder.
    
    Args:
        df: DataFrame with 'id_code' column
        input_dir: Path to raw images (train_images/)
        output_dir: Path to save processed images (processed/)
        preprocessor: BenGrahamPreprocessor instance
        overwrite: If True, reprocess even if output exists
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    success_count = 0
    skipped_count = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="🔄 Preprocessing raw images"):
        img_name = row['id_code']
        output_path = output_dir / f"{img_name}.png"
        
        # Skip if already processed
        if output_path.exists() and not overwrite:
            skipped_count += 1
            success_count += 1
            continue
        
        # Find input image (raw)
        img_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            potential_path = input_dir / f"{img_name}{ext}"
            if potential_path.exists():
                img_path = potential_path
                break
        
        if img_path is None:
            print(f"⚠️ Warning: Raw image not found for {img_name}")
            continue
        
        try:
            img = cv2.imread(str(img_path))
            processed = preprocessor(img)
            cv2.imwrite(str(output_path), processed)
            success_count += 1
        except Exception as e:
            print(f"❌ Error processing {img_name}: {e}")
    
    print(f"\n✅ Preprocessing complete!")
    print(f"   - Processed: {success_count - skipped_count} new images")
    print(f"   - Skipped (already done): {skipped_count} images")
    print(f"   - Total ready: {success_count}/{len(df)} images")
    return success_count


# ============================================
# CHECK IF PREPROCESSING IS NEEDED
# ============================================
input_dir = config.DATA_DIR / config.TRAIN_IMAGES  # Raw images
output_dir = config.DATA_DIR / config.PROCESSED_IMAGES  # Processed output

# Count existing processed images
if output_dir.exists():
    processed_count = len(list(output_dir.glob("*.png")))
else:
    processed_count = 0

# Count raw images
if input_dir.exists():
    raw_count = len(list(input_dir.glob("*.png"))) + len(list(input_dir.glob("*.jpg")))
else:
    raw_count = 0

print(f"📊 Preprocessing Status:")
print(f"   - Raw images (train_images/): {raw_count}")
print(f"   - Processed images (processed/): {processed_count}")

if 'df' in dir():
    total_needed = len(df)
    if processed_count >= total_needed:
        print(f"\n✅ All {total_needed} images already preprocessed! Skipping...")
        RUN_PREPROCESSING = False
    elif processed_count > 0:
        print(f"\n⚠️ Partial preprocessing: {processed_count}/{total_needed} done")
        print("   Set RUN_PREPROCESSING = True to complete preprocessing")
        RUN_PREPROCESSING = True  # Will only process missing images
    else:
        print(f"\n🔄 Need to preprocess {total_needed} images")
        RUN_PREPROCESSING = True
else:
    print("\n⚠️ DataFrame 'df' not found. Load train.csv first!")
    RUN_PREPROCESSING = False


# ============================================
# RUN PREPROCESSING (if needed)
# ============================================
if RUN_PREPROCESSING and 'df' in dir():
    print("\n" + "="*50)
    print("🔧 Starting Ben Graham Preprocessing...")
    print("="*50)
    
    preprocessor = BenGrahamPreprocessor(output_size=config.BEN_GRAHAM_SIZE)
    
    preprocess_all_images(
        df=df,
        input_dir=input_dir,
        output_dir=output_dir,
        preprocessor=preprocessor,
        overwrite=False  # Skip already processed images
    )
    
    print("\n📁 Processed images saved to:", output_dir)

⏭️ Skipping batch preprocessing (set RUN_PREPROCESSING=True to run)
   Images will be preprocessed on-the-fly during training.


---
## ✅ Data Pipeline Complete!

**What happened:**
1. ✅ **Raw images loaded** from `train_images/`
2. ✅ **Ben Graham preprocessing applied** → saved to `processed/`
3. ✅ **Ready for training** with preprocessed images

**Data Flow Summary:**
```
train_images/ (raw) → Ben Graham → processed/ (ready for training)
```

**Next Steps:**
- Model architecture definition
- Data augmentation and loaders
- Training loop
- Evaluation and inference

In [ ]:
# ============================================
# DATA PIPELINE SUMMARY
# ============================================
print("="*60)
print("📊 DATA PIPELINE STATUS")
print("="*60)

# 1. Check CSV
if 'df' in dir():
    print(f"\n1️⃣ Labels (train.csv):")
    print(f"   ✅ Loaded: {len(df)} samples")
    print(f"   Classes: {df['diagnosis'].value_counts().to_dict()}")
else:
    print(f"\n1️⃣ Labels: ❌ Not loaded")

# 2. Check raw images
raw_dir = config.DATA_DIR / config.TRAIN_IMAGES
if raw_dir.exists():
    raw_count = len(list(raw_dir.glob("*.png"))) + len(list(raw_dir.glob("*.jpg")))
    print(f"\n2️⃣ Raw Images (train_images/):")
    print(f"   ✅ Found: {raw_count} images")
else:
    raw_count = 0
    print(f"\n2️⃣ Raw Images: ❌ Not found at {raw_dir}")

# 3. Check processed images
proc_dir = config.DATA_DIR / config.PROCESSED_IMAGES
if proc_dir.exists():
    proc_count = len(list(proc_dir.glob("*.png")))
    print(f"\n3️⃣ Processed Images (processed/):")
    print(f"   ✅ Ready: {proc_count} images")
    
    if 'df' in dir() and proc_count >= len(df):
        print(f"   ✅ All images preprocessed!")
    elif 'df' in dir():
        print(f"   ⚠️ Missing: {len(df) - proc_count} images")
else:
    print(f"\n3️⃣ Processed Images: ❌ Not created yet")
    print(f"   Run preprocessing cell above!")

print("\n" + "="*60)
print("🚀 Data pipeline ready! Continue to Model Architecture →")
print("="*60)

📊 DATA STATUS SUMMARY
❌ Training CSV not loaded
❌ Image directory not found
ℹ️ No preprocessed images (will process on-the-fly)

🚀 Ready for training! Run the next cells to continue.


---
## 4. 🏗️ Model Architecture

The model consists of:
- **Backbone**: EfficientNet-B5 with Noisy Student pretraining
- **Pooling**: Generalized Mean Pooling (GeM) - learnable pooling between avg and max
- **Head**: Regression head for ordinal output (0-4 scale)

This architecture follows the winning solution from APTOS 2019.

In [ ]:
# Generalized Mean Pooling (GeM)
class GeM(nn.Module):
    """
    Generalized Mean Pooling.
    As p → 1: average pooling
    As p → ∞: max pooling
    Default p=3 is a good balance.
    """
    
    def __init__(self, p: float = 3.0, eps: float = 1e-6, trainable: bool = True):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p) if trainable else torch.tensor([p])
        self.eps = eps
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.clamp(min=self.eps)
        pooled = F.adaptive_avg_pool2d(x.pow(self.p), 1)
        return pooled.pow(1.0 / self.p).flatten(1)


# Model Heads
class RegressionHead(nn.Module):
    """Regression head for ordinal output."""
    
    def __init__(self, in_features: int, hidden_dims: List[int] = [512], dropout: float = 0.5):
        super().__init__()
        
        layers = []
        prev_dim = in_features
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        self.head = nn.Sequential(*layers)
        
        # Initialize
        nn.init.xavier_uniform_(self.head[-1].weight)
        nn.init.zeros_(self.head[-1].bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.head(x).squeeze(-1)


print("✅ GeM Pooling and Regression Head defined!")

✅ GeM Pooling and Regression Head defined!


In [ ]:
# Complete DR Model
class DRModel(nn.Module):
    """
    Diabetic Retinopathy Detection Model.
    
    Architecture:
    - EfficientNet backbone (pretrained)
    - GeM pooling
    - Regression head
    """
    
    def __init__(
        self,
        backbone_name: str = "tf_efficientnet_b5_ns",
        pretrained: bool = True,
        pooling: str = "gem",
        head_dropout: float = 0.5,
        hidden_dims: List[int] = [512],
    ):
        super().__init__()
        
        # Create backbone without classifier
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool="",
        )
        
        # Get feature dimension
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            features = self.backbone(dummy)
            self.num_features = features.shape[1]
        
        # Pooling
        if pooling == "gem":
            self.pool = GeM(p=3.0, trainable=True)
        else:
            self.pool = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten())
        
        # Head
        self.head = RegressionHead(
            in_features=self.num_features,
            hidden_dims=hidden_dims,
            dropout=head_dropout,
        )
        
        print(f"Model created: {backbone_name}")
        print(f"  - Feature dimension: {self.num_features}")
        print(f"  - Pooling: {pooling}")
        print(f"  - Head: Regression with dims {hidden_dims}")
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.backbone(x)
        pooled = self.pool(features)
        return self.head(pooled)
    
    def get_features(self, x: torch.Tensor) -> torch.Tensor:
        """Get features before head (for visualization)."""
        features = self.backbone(x)
        return self.pool(features)


# Test model creation
print("\n🧪 Testing model creation...")
test_model = DRModel(backbone_name="efficientnet_b0", pretrained=True)  # Small model for test
test_input = torch.randn(2, 3, 224, 224)
test_output = test_model(test_input)
print(f"✅ Test output shape: {test_output.shape}")
del test_model, test_input, test_output
torch.cuda.empty_cache() if torch.cuda.is_available() else None


🧪 Testing model creation...
Model created: efficientnet_b0
  - Feature dimension: 1280
  - Pooling: gem
  - Head: Regression with dims [512]
✅ Test output shape: torch.Size([2])
Model created: efficientnet_b0
  - Feature dimension: 1280
  - Pooling: gem
  - Head: Regression with dims [512]
✅ Test output shape: torch.Size([2])


---
## 5. 📦 Dataset & Data Augmentation

**The dataset loads preprocessed images from `processed/` folder** (created in Section 3).

If preprocessed images are not found, it will apply Ben Graham preprocessing on-the-fly (slower).

**Augmentations** (designed for retinal fundus images):
- **Geometric**: Rotation (360°), flips - fundus images are rotationally invariant
- **Photometric**: Brightness/contrast adjustments (moderate to preserve pathology colors)
- **Dropout**: Coarse dropout for regularization

In [ ]:
# Data Augmentation Transforms - Optimized for speed
def get_train_transforms(image_size: int = 456) -> A.Compose:
    """Training augmentations for fundus images - optimized for GPU training."""
    return A.Compose([
        # Use INTER_LINEAR (faster than default INTER_AREA for downscaling)
        A.Resize(image_size, image_size, interpolation=cv2.INTER_LINEAR),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=360, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0, 
                 interpolation=cv2.INTER_LINEAR),
        A.RandomScale(scale_limit=0.2, p=0.5, interpolation=cv2.INTER_LINEAR),
        A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.CenterCrop(height=image_size, width=image_size),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.5),
        A.GaussianBlur(blur_limit=(3, 5), p=0.2),
        A.CoarseDropout(num_holes_range=(1, 8), hole_height_range=(20, 50), 
                        hole_width_range=(20, 50), fill=0, p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])


def get_val_transforms(image_size: int = 456) -> A.Compose:
    """Validation transforms (no augmentation) - optimized for speed."""
    return A.Compose([
        A.Resize(image_size, image_size, interpolation=cv2.INTER_LINEAR),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])


print("✅ Transforms defined with optimized interpolation!")

✅ Transforms defined with optimized interpolation!


In [ ]:
# Dataset Class - Optimized for fast loading
class APTOSDataset(Dataset):
    """PyTorch Dataset for APTOS 2019 - Optimized for GPU training."""
    
    def __init__(
        self,
        dataframe: pd.DataFrame,
        image_dir: Path,
        transform: Optional[Callable] = None,
        preprocessor: Optional[BenGrahamPreprocessor] = None,
        use_preprocessed: bool = False,
        preprocessed_dir: Optional[Path] = None,
    ):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.preprocessor = preprocessor
        self.use_preprocessed = use_preprocessed
        self.preprocessed_dir = Path(preprocessed_dir) if preprocessed_dir else None
        
        self.labels = self.df['diagnosis'].values.astype(np.float32)  # Pre-cast to float32
        self.image_ids = self.df['id_code'].values
        
        # Pre-compute image paths for faster access
        self.image_paths = self._precompute_paths()
    
    def _precompute_paths(self) -> List[Path]:
        """Pre-compute all image paths to avoid repeated file system checks."""
        paths = []
        for image_id in self.image_ids:
            if self.use_preprocessed and self.preprocessed_dir:
                path = self.preprocessed_dir / f"{image_id}.png"
            else:
                path = None
                for ext in ['.png', '.jpg', '.jpeg']:
                    potential_path = self.image_dir / f"{image_id}{ext}"
                    if potential_path.exists():
                        path = potential_path
                        break
                if path is None:
                    path = self.image_dir / f"{image_id}.png"  # Default
            paths.append(path)
        return paths
    
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load image using cv2 (faster than PIL for large images)
        image = cv2.imread(str(image_path))
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply Ben Graham preprocessing if not using preprocessed images
        if self.preprocessor and not self.use_preprocessed:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = self.preprocessor(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        else:
            image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
        
        return {
            'image': image,
            'target': torch.tensor(label, dtype=torch.float32),
        }


print("✅ APTOSDataset defined with optimized path caching!")

✅ APTOSDataset defined with optimized path caching!


In [ ]:
# DataModule for PyTorch Lightning - Optimized for GPU Training
class APTOSDataModule(pl.LightningDataModule):
    """PyTorch Lightning DataModule for APTOS dataset - Optimized for fast GPU training."""
    
    def __init__(
        self,
        data_dir: Path,
        train_csv: str,
        image_dir: str,
        processed_dir: Optional[str] = None,
        batch_size: int = 16,
        num_workers: int = 4,
        image_size: int = 456,
        val_split: float = 0.2,
        use_weighted_sampler: bool = True,
        use_preprocessing: bool = True,
        seed: int = 42,
        pin_memory: bool = True,
        persistent_workers: bool = True,
        prefetch_factor: int = 2,
    ):
        super().__init__()
        self.data_dir = Path(data_dir)
        self.train_csv = train_csv
        self.image_dir = image_dir
        self.processed_dir = processed_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.image_size = image_size
        self.val_split = val_split
        self.use_weighted_sampler = use_weighted_sampler
        self.use_preprocessing = use_preprocessing
        self.seed = seed
        
        # GPU optimization settings
        self.pin_memory = pin_memory and torch.cuda.is_available()
        self.persistent_workers = persistent_workers and (num_workers > 0)
        self.prefetch_factor = prefetch_factor if num_workers > 0 else None
        
        self.train_df = None
        self.val_df = None
        self.class_weights = None
    
    def setup(self, stage: Optional[str] = None):
        # Load CSV
        df = pd.read_csv(self.data_dir / self.train_csv)
        
        # Train/val split
        train_idx, val_idx = train_test_split(
            np.arange(len(df)),
            test_size=self.val_split,
            stratify=df['diagnosis'],
            random_state=self.seed
        )
        
        self.train_df = df.iloc[train_idx].reset_index(drop=True)
        self.val_df = df.iloc[val_idx].reset_index(drop=True)
        
        # Calculate class weights
        train_labels = self.train_df['diagnosis'].values
        class_counts = np.bincount(train_labels, minlength=5)
        self.class_weights = 1.0 / np.maximum(class_counts, 1)
        
        # Preprocessor
        preprocessor = BenGrahamPreprocessor(output_size=512) if self.use_preprocessing else None
        
        # Check for preprocessed images
        use_preprocessed = False
        preprocessed_path = None
        if self.processed_dir:
            preprocessed_path = self.data_dir / self.processed_dir
            if preprocessed_path.exists() and len(list(preprocessed_path.glob("*.png"))) > 0:
                use_preprocessed = True
                print(f"✅ Using preprocessed images from {preprocessed_path}")
            else:
                print("ℹ️ Preprocessed images not found, will process on-the-fly")
        
        # Create datasets
        self.train_dataset = APTOSDataset(
            dataframe=self.train_df,
            image_dir=self.data_dir / self.image_dir,
            transform=get_train_transforms(self.image_size),
            preprocessor=preprocessor if not use_preprocessed else None,
            use_preprocessed=use_preprocessed,
            preprocessed_dir=preprocessed_path,
        )
        
        self.val_dataset = APTOSDataset(
            dataframe=self.val_df,
            image_dir=self.data_dir / self.image_dir,
            transform=get_val_transforms(self.image_size),
            preprocessor=preprocessor if not use_preprocessed else None,
            use_preprocessed=use_preprocessed,
            preprocessed_dir=preprocessed_path,
        )
        
        print(f"📊 Train samples: {len(self.train_dataset)}, Val samples: {len(self.val_dataset)}")
        print(f"⚡ GPU Optimizations: pin_memory={self.pin_memory}, persistent_workers={self.persistent_workers}, prefetch_factor={self.prefetch_factor}")
    
    def _get_sampler(self):
        if not self.use_weighted_sampler:
            return None
        
        train_labels = self.train_df['diagnosis'].values
        sample_weights = self.class_weights[train_labels]
        
        return WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )
    
    def train_dataloader(self):
        sampler = self._get_sampler()
        loader_kwargs = {
            'batch_size': self.batch_size,
            'shuffle': (sampler is None),
            'sampler': sampler,
            'num_workers': self.num_workers,
            'pin_memory': self.pin_memory,
            'drop_last': True,
        }
        # Only add these if num_workers > 0
        if self.num_workers > 0:
            loader_kwargs['persistent_workers'] = self.persistent_workers
            loader_kwargs['prefetch_factor'] = self.prefetch_factor
        
        return DataLoader(self.train_dataset, **loader_kwargs)
    
    def val_dataloader(self):
        loader_kwargs = {
            'batch_size': self.batch_size * 2,  # Larger batch for validation (no gradients)
            'shuffle': False,
            'num_workers': self.num_workers,
            'pin_memory': self.pin_memory,
        }
        # Only add these if num_workers > 0
        if self.num_workers > 0:
            loader_kwargs['persistent_workers'] = self.persistent_workers
            loader_kwargs['prefetch_factor'] = self.prefetch_factor
        
        return DataLoader(self.val_dataset, **loader_kwargs)


print("✅ APTOSDataModule defined with GPU optimizations!")

✅ APTOSDataModule defined with GPU optimizations!


---
## 6. 📊 Metrics & Threshold Optimization

The primary metric is **Quadratic Weighted Kappa (QWK)** - measures agreement between predicted and actual grades with quadratic penalty for distance.

Key technique: **Threshold Optimization** - find optimal thresholds to convert regression output to discrete classes.

In [ ]:
# Metrics
def quadratic_weighted_kappa(y_true: np.ndarray, y_pred: np.ndarray, num_classes: int = 5) -> float:
    """Calculate Quadratic Weighted Kappa."""
    y_true = np.clip(np.asarray(y_true).astype(int), 0, num_classes - 1)
    y_pred = np.clip(np.asarray(y_pred).astype(int), 0, num_classes - 1)
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


# Threshold Optimizer
class ThresholdOptimizer:
    """Optimize thresholds to maximize QWK."""
    
    def __init__(self, num_classes: int = 5, initial_thresholds: List[float] = None):
        self.num_classes = num_classes
        self.initial_thresholds = initial_thresholds or [0.5, 1.5, 2.5, 3.5]
    
    def _apply_thresholds(self, predictions: np.ndarray, thresholds: List[float]) -> np.ndarray:
        """Convert continuous predictions to classes."""
        classes = np.zeros(len(predictions), dtype=int)
        for i, threshold in enumerate(thresholds):
            classes[predictions > threshold] = i + 1
        return classes
    
    def _objective(self, thresholds: np.ndarray, predictions: np.ndarray, targets: np.ndarray) -> float:
        """Objective: negative QWK (for minimization)."""
        thresholds = np.sort(thresholds)
        pred_classes = self._apply_thresholds(predictions, thresholds)
        return -quadratic_weighted_kappa(targets, pred_classes, self.num_classes)
    
    def optimize(self, predictions: np.ndarray, targets: np.ndarray, verbose: bool = True) -> Tuple[List[float], float]:
        """Find optimal thresholds."""
        predictions = predictions.flatten()
        targets = targets.flatten().astype(int)
        
        # Initial QWK
        initial_qwk = -self._objective(np.array(self.initial_thresholds), predictions, targets)
        if verbose:
            print(f"Initial QWK with thresholds {self.initial_thresholds}: {initial_qwk:.4f}")
        
        # Optimize
        result = minimize(
            self._objective,
            x0=np.array(self.initial_thresholds),
            args=(predictions, targets),
            method='Nelder-Mead',
            options={'maxiter': 1000}
        )
        
        optimized_thresholds = sorted(result.x.tolist())
        best_qwk = -result.fun
        
        if verbose:
            print(f"Optimized QWK: {best_qwk:.4f}")
            print(f"Optimized thresholds: {[f'{t:.3f}' for t in optimized_thresholds]}")
        
        return optimized_thresholds, best_qwk
    
    def apply(self, predictions: np.ndarray, thresholds: List[float] = None) -> np.ndarray:
        """Apply thresholds to predictions."""
        thresholds = thresholds or self.initial_thresholds
        return self._apply_thresholds(predictions.flatten(), thresholds)


print("✅ Metrics and ThresholdOptimizer defined!")

✅ Metrics and ThresholdOptimizer defined!


---
## 7. ⚡ PyTorch Lightning Training Module

The training module handles:
- Forward pass and loss computation
- Metric logging (train/val loss, QWK)
- Optimizer and scheduler configuration
- Threshold optimization at epoch end

In [ ]:
# PyTorch Lightning Training Module
class DRTrainingModule(pl.LightningModule):
    """PyTorch Lightning module for DR detection training."""
    
    def __init__(
        self,
        backbone_name: str = "tf_efficientnet_b5_ns",
        pretrained: bool = True,
        learning_rate: float = 1e-4,
        weight_decay: float = 1e-5,
        num_epochs: int = 30,
        optimize_thresholds: bool = True,
    ):
        super().__init__()
        self.save_hyperparameters()
        
        # Model
        self.model = DRModel(
            backbone_name=backbone_name,
            pretrained=pretrained,
            pooling="gem",
            head_dropout=0.5,
        )
        
        # Loss
        self.criterion = nn.MSELoss()
        
        # Training params
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs
        self.optimize_thresholds = optimize_thresholds
        
        # Threshold optimizer
        self.threshold_optimizer = ThresholdOptimizer()
        self.optimized_thresholds = [0.5, 1.5, 2.5, 3.5]
        
        # Store outputs for epoch-level metrics
        self.training_step_outputs = []
        self.validation_step_outputs = []
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        images = batch['image']
        targets = batch['target']
        
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        
        # Store for epoch metrics
        preds = outputs.round().clamp(0, 4).long()
        self.training_step_outputs.append({
            'loss': loss.detach(),
            'preds': preds.detach().cpu(),
            'targets': targets.long().detach().cpu(),
        })
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        if self.training_step_outputs:
            all_preds = torch.cat([x['preds'] for x in self.training_step_outputs])
            all_targets = torch.cat([x['targets'] for x in self.training_step_outputs])
            
            qwk = quadratic_weighted_kappa(all_targets.numpy(), all_preds.numpy())
            self.log('train_qwk', qwk, prog_bar=True)
        
        self.training_step_outputs.clear()
    
    def validation_step(self, batch, batch_idx):
        images = batch['image']
        targets = batch['target']
        
        outputs = self(images)
        loss = self.criterion(outputs, targets)
        
        self.validation_step_outputs.append({
            'loss': loss.detach(),
            'outputs': outputs.detach().cpu(),
            'targets': targets.long().detach().cpu(),
        })
        
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self):
        if self.validation_step_outputs:
            all_outputs = torch.cat([x['outputs'] for x in self.validation_step_outputs])
            all_targets = torch.cat([x['targets'] for x in self.validation_step_outputs])
            
            # QWK with default thresholds
            preds = all_outputs.round().clamp(0, 4).long()
            qwk = quadratic_weighted_kappa(all_targets.numpy(), preds.numpy())
            
            # Optimize thresholds
            if self.optimize_thresholds:
                opt_thresholds, opt_qwk = self.threshold_optimizer.optimize(
                    all_outputs.numpy(),
                    all_targets.numpy(),
                    verbose=False
                )
                self.optimized_thresholds = opt_thresholds
                self.log('val_qwk_optimized', opt_qwk, prog_bar=True)
            
            self.log('val_qwk', qwk, prog_bar=True)
            
            # Accuracy
            accuracy = (preds == all_targets).float().mean().item()
            self.log('val_accuracy', accuracy, prog_bar=True)
        
        self.validation_step_outputs.clear()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay,
        )
        
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=self.num_epochs,
            eta_min=self.learning_rate * 0.01,
        )
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {'scheduler': scheduler, 'interval': 'epoch'}
        }


print("✅ DRTrainingModule defined!")

✅ DRTrainingModule defined!


---
## 8. 🚀 Training

Now let's train the model!

In [ ]:
# Create DataModule with GPU optimizations
datamodule = APTOSDataModule(
    data_dir=config.DATA_DIR,
    train_csv=config.TRAIN_CSV,
    image_dir=config.TRAIN_IMAGES,
    processed_dir=config.PROCESSED_IMAGES,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    image_size=config.IMAGE_SIZE,
    val_split=config.VAL_SPLIT,
    use_weighted_sampler=True,
    use_preprocessing=config.USE_PREPROCESSING,
    seed=config.SEED,
    # GPU optimization parameters
    pin_memory=config.PIN_MEMORY,
    persistent_workers=config.PERSISTENT_WORKERS,
    prefetch_factor=config.PREFETCH_FACTOR,
)

# Setup datamodule (this loads and splits the data)
datamodule.setup()

print(f"\n📊 DataModule ready!")
print(f"   Train batches: {len(datamodule.train_dataloader())}")
print(f"   Val batches: {len(datamodule.val_dataloader())}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/aptos/train.csv'

In [ ]:
# Visualize a batch
batch = next(iter(datamodule.train_dataloader()))
images = batch['image']
targets = batch['target']

print(f"Batch shape: {images.shape}")
print(f"Targets: {targets}")

# Denormalize and show
mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i, ax in enumerate(axes.flatten()):
    if i < len(images):
        img = images[i].cpu() * std + mean
        img = img.permute(1, 2, 0).numpy()
        img = np.clip(img, 0, 1)
        ax.imshow(img)
        ax.set_title(f"Grade: {int(targets[i])} ({config.CLASS_NAMES[int(targets[i])]})")
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Create Lightning Module
model = DRTrainingModule(
    backbone_name=config.BACKBONE,
    pretrained=True,
    learning_rate=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    num_epochs=config.EPOCHS,
    optimize_thresholds=True,
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Model Summary:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

In [ ]:
# GPU Warmup and Memory Check
if torch.cuda.is_available():
    print("🔥 Warming up GPU...")
    
    # Show GPU info
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"   GPU: {gpu_name}")
    print(f"   Total Memory: {gpu_memory:.2f} GB")
    
    # Warmup: run a few forward passes to initialize CUDA kernels
    dummy_input = torch.randn(4, 3, config.IMAGE_SIZE, config.IMAGE_SIZE).cuda()
    with torch.no_grad():
        for _ in range(3):
            _ = torch.nn.functional.conv2d(dummy_input, torch.randn(64, 3, 7, 7).cuda())
    
    del dummy_input
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    # Memory after warmup
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"   Memory Allocated: {allocated:.2f} GB")
    print(f"   Memory Reserved: {reserved:.2f} GB")
    print("✅ GPU warmup complete!")
else:
    print("⚠️ CUDA not available - running on CPU (will be slow)")

In [ ]:
# Setup callbacks
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_dir = config.CHECKPOINT_DIR / timestamp
checkpoint_dir.mkdir(parents=True, exist_ok=True)

callbacks = [
    ModelCheckpoint(
        dirpath=checkpoint_dir,
        filename="dr-{epoch:02d}-{val_qwk:.4f}",
        monitor="val_qwk",
        mode="max",
        save_top_k=3,
        save_last=True,
        verbose=True,
    ),
    EarlyStopping(
        monitor="val_qwk",
        mode="max",
        patience=config.PATIENCE,
        verbose=True,
    ),
    LearningRateMonitor(logging_interval="epoch"),
]

# Loggers
loggers = [
    TensorBoardLogger(save_dir=config.LOG_DIR, name="tensorboard"),
    CSVLogger(save_dir=config.LOG_DIR, name="csv_logs"),
]

print(f"✅ Callbacks and loggers configured!")
print(f"   Checkpoints will be saved to: {checkpoint_dir}")

In [ ]:
# Create Trainer with GPU optimizations
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    accelerator="auto",  # Automatically detect GPU/CPU
    devices=1,
    precision=config.PRECISION,  # 16-mixed for faster training
    callbacks=callbacks,
    logger=loggers,
    accumulate_grad_batches=config.ACCUMULATE_GRAD,
    gradient_clip_val=1.0,
    log_every_n_steps=10,
    enable_progress_bar=True,
    deterministic=False,  # False for speed (cuDNN benchmark enabled)
    benchmark=True,  # Enable cuDNN auto-tuner for faster convolutions
)

print("✅ Trainer configured!")
print(f"   Max epochs: {config.EPOCHS}")
print(f"   Precision: {config.PRECISION}")
print(f"   Accumulate gradients: {config.ACCUMULATE_GRAD}")
print(f"   Effective batch size: {config.BATCH_SIZE * config.ACCUMULATE_GRAD}")
print(f"   cuDNN benchmark: Enabled")
print(f"   Accelerator: {trainer.accelerator}")

In [ ]:
# Clear GPU cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("🧹 GPU cache cleared - ready for training!")
    
    # Display available memory
    free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved(0)
    print(f"   Available GPU memory: {free_memory / 1e9:.2f} GB")

In [ ]:
# 🚀 START TRAINING
print("="*60)
print("🚀 STARTING TRAINING")
print("="*60)
print(f"Device: {DEVICE}")
print(f"Epochs: {config.EPOCHS}")
print(f"Batch size: {config.BATCH_SIZE} (effective: {config.BATCH_SIZE * config.ACCUMULATE_GRAD})")
print(f"Learning rate: {config.LEARNING_RATE}")
print(f"Backbone: {config.BACKBONE}")
print(f"Mixed Precision: {config.PRECISION}")
print(f"Workers: {config.NUM_WORKERS} (pin_memory={config.PIN_MEMORY})")
print("="*60)

# Measure training time
start_time = time.time()

trainer.fit(model, datamodule=datamodule)

end_time = time.time()
training_time = end_time - start_time

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print(f"⏱️ Total training time: {training_time/60:.2f} minutes")
print("="*60)

---
## 9. 📈 Evaluation & Results

In [ ]:
# Training Results
print("📊 Training Results")
print("="*60)

# Best model info
best_model_path = trainer.checkpoint_callback.best_model_path
best_qwk = trainer.checkpoint_callback.best_model_score

print(f"Best validation QWK: {best_qwk:.4f}")
print(f"Best model saved to: {best_model_path}")
print(f"Optimized thresholds: {model.optimized_thresholds}")

In [ ]:
# Load best model and evaluate
best_model = DRTrainingModule.load_from_checkpoint(best_model_path)
best_model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)

# Run validation with GPU optimizations
all_preds = []
all_targets = []
all_outputs = []

with torch.no_grad(), torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
    for batch in tqdm(datamodule.val_dataloader(), desc="Evaluating"):
        # Use non_blocking=True for async GPU transfer when using pinned memory
        images = batch['image'].to(device, non_blocking=True)
        targets = batch['target']
        
        outputs = best_model(images)
        
        all_outputs.extend(outputs.cpu().numpy())
        all_targets.extend(targets.numpy())

all_outputs = np.array(all_outputs)
all_targets = np.array(all_targets).astype(int)

# Apply optimized thresholds
threshold_optimizer = ThresholdOptimizer()
opt_thresholds, opt_qwk = threshold_optimizer.optimize(all_outputs, all_targets)
all_preds = threshold_optimizer.apply(all_outputs, opt_thresholds)

print(f"\n✅ Final Validation QWK: {opt_qwk:.4f}")

In [ ]:
# Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw confusion matrix
cm = confusion_matrix(all_targets, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=config.CLASS_NAMES, yticklabels=config.CLASS_NAMES)
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix')

# Normalized confusion matrix
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=config.CLASS_NAMES, yticklabels=config.CLASS_NAMES)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Normalized Confusion Matrix')

plt.tight_layout()
plt.show()

# Classification report
print("\n📊 Classification Report:")
print(classification_report(all_targets, all_preds, target_names=config.CLASS_NAMES))

In [ ]:
# Plot training history (from CSV logs)
csv_log_dir = config.LOG_DIR / "csv_logs"
versions = sorted(csv_log_dir.glob("version_*"))

if versions:
    latest_version = versions[-1]
    metrics_file = latest_version / "metrics.csv"
    
    if metrics_file.exists():
        metrics_df = pd.read_csv(metrics_file)
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Loss
        if 'train_loss_epoch' in metrics_df.columns:
            train_loss = metrics_df.dropna(subset=['train_loss_epoch'])
            axes[0].plot(train_loss['epoch'], train_loss['train_loss_epoch'], label='Train', marker='o')
        if 'val_loss' in metrics_df.columns:
            val_loss = metrics_df.dropna(subset=['val_loss'])
            axes[0].plot(val_loss['epoch'], val_loss['val_loss'], label='Validation', marker='o')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Training & Validation Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # QWK
        if 'train_qwk' in metrics_df.columns:
            train_qwk = metrics_df.dropna(subset=['train_qwk'])
            axes[1].plot(train_qwk['epoch'], train_qwk['train_qwk'], label='Train', marker='o')
        if 'val_qwk' in metrics_df.columns:
            val_qwk = metrics_df.dropna(subset=['val_qwk'])
            axes[1].plot(val_qwk['epoch'], val_qwk['val_qwk'], label='Validation', marker='o')
        if 'val_qwk_optimized' in metrics_df.columns:
            val_qwk_opt = metrics_df.dropna(subset=['val_qwk_optimized'])
            axes[1].plot(val_qwk_opt['epoch'], val_qwk_opt['val_qwk_optimized'], 
                        label='Val (optimized)', marker='s', linestyle='--')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('QWK')
        axes[1].set_title('Quadratic Weighted Kappa')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # Learning rate
        if 'lr-AdamW' in metrics_df.columns:
            lr_data = metrics_df.dropna(subset=['lr-AdamW'])
            axes[2].plot(lr_data['epoch'], lr_data['lr-AdamW'], marker='o', color='green')
        axes[2].set_xlabel('Epoch')
        axes[2].set_ylabel('Learning Rate')
        axes[2].set_title('Learning Rate Schedule')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ No training logs found")

---
## 10. 💾 Save & Export Model

In [ ]:
# Save model info for inference
model_info = {
    'backbone': config.BACKBONE,
    'image_size': config.IMAGE_SIZE,
    'num_classes': config.NUM_CLASSES,
    'optimized_thresholds': opt_thresholds,
    'best_qwk': float(opt_qwk),
    'checkpoint_path': str(best_model_path),
}

import json
info_path = checkpoint_dir / 'model_info.json'
with open(info_path, 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"✅ Model info saved to: {info_path}")
print(json.dumps(model_info, indent=2))

In [ ]:
# Export to ONNX (optional)
def export_to_onnx(model, output_path, image_size=456):
    """Export model to ONNX format."""
    model.eval()
    device = next(model.parameters()).device
    
    dummy_input = torch.randn(1, 3, image_size, image_size).to(device)
    
    torch.onnx.export(
        model.model,  # The actual model (not Lightning module)
        dummy_input,
        output_path,
        export_params=True,
        opset_version=14,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    )
    print(f"✅ ONNX model exported to: {output_path}")

# Uncomment to export
# onnx_path = checkpoint_dir / 'model.onnx'
# export_to_onnx(best_model, onnx_path, config.IMAGE_SIZE)

In [ ]:
# Download model files (for Colab)
if IN_COLAB:
    from google.colab import files
    
    # Create zip of checkpoint directory
    import shutil
    zip_path = f'/content/dr_model_{timestamp}.zip'
    shutil.make_archive(zip_path.replace('.zip', ''), 'zip', checkpoint_dir)
    
    print(f"📦 Model zip created: {zip_path}")
    print("Click below to download:")
    files.download(zip_path)
else:
    print(f"📁 Model saved locally at: {checkpoint_dir}")

---
## 11. 🔮 Inference Demo

Quick demonstration of how to use the trained model for inference.

In [ ]:
# Inference function
def predict_single_image(model, image_path, preprocessor, transform, thresholds, device='cuda'):
    """Predict DR grade for a single image."""
    # Load and preprocess
    img = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Ben Graham preprocessing
    processed = preprocessor(img)
    processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    
    # Transform
    transformed = transform(image=processed_rgb)
    tensor = transformed['image'].unsqueeze(0).to(device)
    
    # Predict
    model.eval()
    with torch.no_grad():
        output = model(tensor).cpu().numpy()[0]
    
    # Apply thresholds
    pred_class = 0
    for i, t in enumerate(thresholds):
        if output > t:
            pred_class = i + 1
    
    return {
        'raw_output': float(output),
        'predicted_class': pred_class,
        'class_name': config.CLASS_NAMES[pred_class],
        'original_image': img_rgb,
        'processed_image': processed_rgb,
    }

print("✅ Inference function defined!")

In [ ]:
# Demo: predict on validation samples
preprocessor = BenGrahamPreprocessor(output_size=512)
transform = get_val_transforms(config.IMAGE_SIZE)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get some samples from each class
fig, axes = plt.subplots(5, 3, figsize=(15, 25))

for class_idx in range(5):
    # Get a validation sample from this class
    val_samples = datamodule.val_df[datamodule.val_df['diagnosis'] == class_idx]
    if len(val_samples) > 0:
        sample = val_samples.iloc[0]
        img_name = sample['id_code']
        
        # Find image
        img_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            potential = config.DATA_DIR / config.TRAIN_IMAGES / f"{img_name}{ext}"
            if potential.exists():
                img_path = potential
                break
        
        if img_path:
            result = predict_single_image(
                best_model, img_path, preprocessor, transform, 
                opt_thresholds, device
            )
            
            # Original
            axes[class_idx, 0].imshow(result['original_image'])
            axes[class_idx, 0].set_title(f"Original\\nTrue: {config.CLASS_NAMES[class_idx]}")
            axes[class_idx, 0].axis('off')
            
            # Processed
            axes[class_idx, 1].imshow(result['processed_image'])
            axes[class_idx, 1].set_title(f"Processed")
            axes[class_idx, 1].axis('off')
            
            # Prediction
            color = 'green' if result['predicted_class'] == class_idx else 'red'
            axes[class_idx, 2].text(0.5, 0.6, f"Predicted: {result['class_name']}", 
                                    ha='center', va='center', fontsize=16, color=color,
                                    transform=axes[class_idx, 2].transAxes)
            axes[class_idx, 2].text(0.5, 0.4, f"Raw: {result['raw_output']:.3f}", 
                                    ha='center', va='center', fontsize=12,
                                    transform=axes[class_idx, 2].transAxes)
            axes[class_idx, 2].axis('off')

plt.tight_layout()
plt.show()

---
## 📝 Summary

This notebook provides a complete end-to-end pipeline for Diabetic Retinopathy detection:

### What was implemented:
1. **Data Loading** - APTOS 2019 dataset with visualization
2. **Ben Graham Preprocessing** - Industry-standard fundus image preprocessing
3. **Model Architecture** - EfficientNet-B5 with GeM pooling and regression head
4. **Data Augmentation** - Rotation-invariant augmentations for fundus images
5. **Training** - PyTorch Lightning with early stopping and checkpointing
6. **Threshold Optimization** - Maximize QWK by finding optimal thresholds
7. **Evaluation** - Confusion matrix and classification report
8. **Export** - Save model for deployment

### Key Techniques:
- **Regression approach** instead of classification (ordinal nature of DR grades)
- **Threshold optimization** to convert regression to classes
- **Weighted sampling** to handle class imbalance
- **GeM Pooling** for better feature aggregation
- **Cosine annealing** learning rate schedule

### Next Steps:
- Try different backbones (EfficientNet-B6, B7)
- Implement Test-Time Augmentation (TTA)
- Add Grad-CAM visualization for explainability
- Fine-tune on additional datasets (EyePACS)